# GORG classifier

We developed the GORG (Global Oceans Reference Genomes) classifier to recruit reads to a database made up of single cell genomes and assign taxonomy and function to reads. 

We have already gone ahead and recruited the same reads from metagenomes that we used this morning to the GORG database, [described here](https://www.sciencedirect.com/science/article/pii/S0092867419312735).

The outputs from the GORG-classifier_setup notebook are in this folder storage/lesson_analyses/gorg-classifier

Lets take a look in that folder now.

In [ ]:
!ls /mnt/storage/lesson_analyses/gorg-classifier

Let's first examine the summaries folder.

In [ ]:
!ls /mnt/storage/lesson_analyses/gorg-classifier/summaries

There is a summary for each metagenome run through the GORG-classifier.  These are short files meant to give a very quick assessment of how the GORG-classifier did in assigning reads to taxonomic and functional categories.

The metagenome SRR6507279 is from the same location and timepoint the SAG plate we've been looking at in previous lessons, AG-910. Let's look at this one first.  

Note that we are recruiting to a subset of 5,536 SAGs from the GORG-tropics dataset and not just the AG-910 plate that we have been using in previous lessons. The summaries file produces a nice short text file that is only 12 lines long so by increasing the number of lines that we use in the head command we can look at the entire file at once.

In [ ]:
! head /mnt/storage/lesson_analyses/gorg-classifier/summaries/SRR6507279_summary.txt -n 15

Now lets look at the annotations file to see what we have in there.

Wow! we are recruiting nearly 77% of all of the reads just to gene sequences and most of these reads (61% of the total) have functional annotations to Non-hypothetical genes. 

If you recall from this morning, we were only able to recruit a maximum of around 30% of metagenomic reads to SAGs from plate AG-910.  Expanding the reference database to GORG-Tropics, which uses a curated collection of 5,536 SAGs from the 12,715 cell collection, allowed us to recruit a much greater number of metagenomic reads overall.  

The real meat of the results are in the **annotations** output directory.  Let's explore.

In [ ]:
! ls /mnt/storage/lesson_analyses/gorg-classifier/annotations/

We can look at the first 10 rows of these compressed files like so:

In [ ]:
!zcat /mnt/storage/lesson_analyses/gorg-classifier/annotations/SRR5720233_annotated.txt.gz | head -n10

Alright there is a lot going on there and the formatting is a bit strange so lets go ahead and load all of our libraries and load this file into a dataframe to make it a bit easier to interpret

In [ ]:
import pandas as pd
import os
import sys
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import time

# modify pandas variabls to only show 10 rows.
pd.set_option("display.max_rows", 10)

# change default figure size 
plt.rcParams["figure.figsize"] = (8,6)

# create output directories
outdir = "./tbls/"
figdir = "./figs/"

if not os.path.exists(outdir):
    os.mkdir(outdir)
if not os.path.exists(figdir):
    os.mkdir(figdir)


annotations_file = '/mnt/storage/lesson_analyses/gorg-classifier/annotations/SRR6507279_annotated.txt.gz'

In [ ]:
data=pd.read_csv(annotations_file, compression='gzip', sep='\t')
data.head()

This file contains a row for **every single read** within a metagenome.  You can imagine how these can get pretty massive, and manipulating them in a notebook could get unweildy.  For this lesson, the metagenomes we're using were subsampled to 1 million reads, so these tables are manageable.

OK there is a lot going on here lets break this down a little bit by looking at each of the important columns again
 - status: Tells us if the read was Classified (C) or Unclassified (U)
 - sequence_id: read name
 - taxonomy_id: NCBI tax_ID number. Indicates the NCBI taxonomy
 - taxonomy_ids_lca: NCBI tax_ID number of the last common ancestor if there are multiple matches. This is used to determine the taxonomic assignment
 - taxonomy_lineage: NCBI formatted taxonomy represented by the tax_ID number.
 - The rest of the columns tell us information about the protein using the Prokka annotations


### Important Note:
The taxonomic id here uses the NCBI based taxonomy and not the GTDB based taxonomy that we discussed yesterday. As a result these taxonomies are not directly compatable so it is challenging to try and get them to work together. We are in the process of updating the GORG classifier database so that the GTDB taxonomy will be available as a database as well as some other modifications to make it more user friendly, flexible, and provide a more informative output. Be on the lookout for an update to this program sometime soon

## Functional characterization of metagenomic reads.

Let's look at the functional profiles of this metagenome by taxonomic group. I am going to group this dataframe by the ```taxonomic_lineage``` and ```prokka_product``` columns.

In [ ]:
# I choose to group on the lineage and the prokka_product but you can pick any columns that you want to. The count column (sequence_id)
# needs to be one of the columns that you are not grouping by but other than that it can be anything.
grouped=data[data['status']=='C'].groupby(['taxonomic_lineage', 'prokka_product'], as_index= False)['sequence_id'].count()

grouped=grouped.rename(columns={'sequence_id':'number_of_reads'})

In [ ]:
# lets start by looking at the most abundant functions by taxnomic group that are in this dataset

grouped.sort_values(['number_of_reads'], inplace=True, ascending=False)
grouped.head()

Not surprisingly the most abundant proteins are hypothetical proteins since we grouped all hypothetical proteins together in our groupby command. Lets get rid of those proteins so that we can look at the proteins that have a functional annotation and plot them.

In [ ]:
# select the 20 most abundant proteins that are not hypothetical proteins
sub_group=grouped[grouped['prokka_product']!= 'hypothetical protein'][:20]

# genrate the plot 
sns.barplot(data=sub_group, x='prokka_product', y='number_of_reads', hue='taxonomic_lineage', dodge=False)

# create axis labels and rotate axis
plt.xlabel('gene')
plt.ylabel('Number of reads recruited')
plt.title('Most abundant genes in SRR6507279')
plt.xticks(rotation=90)

# move legend outside of chart
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.savefig(os.path.join(figdir,"read_recruitment_to_most_abundant_functions.pdf"))

Most of the most abundant genes are from Pelagibacteraceae. Not too surprising, but there are a couple of genes from either Flavobacteriaceae or Gammaproteobacteria.

Let's next look at the most abundant gene functions overall regardless of taxnomic assignment:

In [ ]:

# group by just the functional annotation to look at functional abundance
functional_df=data[data['status']=='C'].groupby('prokka_product', as_index= False)['sequence_id'].count()

#rename the sequence_id and sort
functional_df=functional_df.rename(columns={'sequence_id':'number_of_reads'})
functional_df.sort_values(['number_of_reads'], inplace=True, ascending=False)

# extract the first 20 rows for the plot
sub_functional_df=functional_df[~functional_df['prokka_product'].isin(['hypothetical protein', 'putative protein'])][:20]

# generate the plot
sub_functional_df.plot(kind='bar', x='prokka_product', y='number_of_reads')

# generate labels
plt.xlabel('functional annotation')
plt.ylabel('number of reads recruited')
plt.title('number of reads from SRR6507279 recruited to functional annotation')

# move legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# save plot
plt.savefig(os.path.join(figdir,"most_abundant_functions_in_SRR6507279.pdf"))


Cool so it looks like DNA directed RNA polymerases are the most common genes in this metagenome. 

Note that the two most abundant genes are essentially the same thing but that one of them has an ' at the very end and the other one does not. As a result of this the computer is identifying them as two different genes. 

It looks like there most of the most common genes are involved in imformation processing (RNA transcriptin, DNA replication, tRNA ligases, DNA repair or DNA synthesis) in some way. 

## Taxonomic characterization of metagenomic reads

Next, I am curious what taxonomic lineages are represented within this metagenome, so I will group by taxonomic lineage, and count reads belonging to each.

In [ ]:
#group by just the taxonomic lineage to look at lineage abundance
tax_df=data[data['status']=='C'].groupby('taxonomic_lineage', as_index= False)['sequence_id'].count()

# rename the sequence id to reflect that it is the number of reads and sort high to low
tax_df=tax_df.rename(columns={'sequence_id':'number_of_reads'})
tax_df.sort_values(['number_of_reads'], inplace=True, ascending=False)

# extract the first 20 rows for the plot
sub_tax_df=tax_df[:20]
# split the classification so that we can use taxonomic levels to color the barchart
sub_tax_df[['domain', 'phyla', 'class', 'order', 'family', 'genus', 'species', '']]=sub_tax_df.taxonomic_lineage.str.split(';', expand=True)


# generate the plot. I will plot the log10 transformation of the number of reads in order to better visualize these values
ax = sns.barplot(data= sub_tax_df, x='taxonomic_lineage', y = 'number_of_reads', hue='order', dodge=False)

# create axis labels 
ax.set(xlabel = 'taxonomic lineage',
       ylabel = 'Number of reads recruited',
       title = 'Number of reads from SRR6507279 recruited to genera', 
       yscale = 'log')

# rotate x-axis labels
plt.xticks(rotation=90)


# move legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.savefig(os.path.join(figdir,"most_abundant_lineages_in_SRR6507279.pdf"))


<br>

## 30 minute coffee and coding break

During this time get some coffee and snacks if you would like. 

Additionally everyone pick a gene of interest and a taxonomic lineage of interest and determine:

For the gene: in what lineage is it most abundant?  

For the lineage: what functional genes are most abundant?  

I'll leave some space below in this notebook for you to work on this.
<br>

When we come back we will look into the distribution of different functional categories in different lineages.

## Examining the distribution of COGs within a metagenome

The annotation output files provide more than just a taxonomic assignment and prokka function. If a gene is aligned to the database, GORG-classifier provides an array of different functional characterizations.  Those include:

* prokka_gene
* prokka_EC_number
* prokka_product
* swissprot_gene
* swissprot_EC_number
* swissprot_product
* swissprot_eggNOG
* swissprot_KO
* swissprot_Pfam
* swissprot_CAZy
* swissprot_TIGRFAMs

These may look familiar to you as they are all flavors of annotaton provided in the standard SCGC SAG deliverable that Ramunas and Greg reviewed on day 1.

Next, I'm going to look into the funcitonal profiles of this metagenome using the COG IDs provided in the ```swissprot_eggNOG``` column.


To do this, I've manipulated the file ```tbls/cog-20.def.csv``` available on the [COG website](https://ftp.ncbi.nih.gov/pub/COG/COG2020/data/).  My manipulated table: ```tbls/cog-20_unambiguous.csv``` has eliminated COGs that are assigned to multiple functional categories.

In [ ]:
# I generated this file from the cog-20.def.csv file that is available for download on the COG website. 
# the major difference is that I eliminated COGs that are assigned to multiple functional categories
COG_summary=pd.read_csv('tbls/cog-20_unambiguous.csv', index_col=0)
# Only keep the columns that we are interested in.
COG_summary=COG_summary[['COG_ID', 'Functional_category', 'COG_name', 'Functional category description']].copy()
COG_summary.head()

Next, I'm going to group my annotation table by ```taxonomic_lineage``` and ```swissprot_eggNOG``` columns, and count how many reads fell into each grouping.

In [ ]:
COG_df=data[data['status']=='C'].groupby(['taxonomic_lineage', 'swissprot_eggNOG'], as_index= False)['sequence_id'].count()

COG_df=COG_df.rename(columns={'sequence_id':'number_of_reads'})
COG_df.sort_values(['number_of_reads'], inplace=True, ascending=False)
COG_df.head(10)

Above you can see that the ```swissprot_eggNOG``` column contains two codes, an EGGNOG id and a CCOG id, separated by a ','.  I'm going to split that column into three columns, as some have more than one COG listed. I'm going to only consider the first listed COG.

I'm also going to correct the COG ID by removing the first 'C'.  Not sure why swissprot has added that additional 'C'.

In [ ]:
# split the eggNOG/COG column so that we can use the COG. There are several genes that have multiple COGs assigned to them. 
# For the purposes of this exercise we are just going to use the first one
COG_df[['eggNOG', 'COG_ID', 'addl_COGs']]=COG_df['swissprot_eggNOG'].str.split(',', 2, expand=True)

#for some reason eggNOG adds an extra "C" to the beginning of their COG designation so we need to remove that
COG_df['COG_ID']=COG_df['COG_ID'].str[1:]
COG_df.head()

Finallly, I'm going only keep the COG_ID column, and merge it with the COG_summary table we'd loaded at the beginning of this section.

In [ ]:
COG_df = COG_df[['COG_ID','taxonomic_lineage', 'number_of_reads']].merge(COG_summary, on='COG_ID', how='left')

COG_df.head()

Now the fun part... let's group the COGs by their taxonomic lineage and the COG Functional category.  This time we need to sum the number of reads instead of counting them, since they'd already been counted within the previous set of transofrmations.

In [ ]:
func_cat_df=COG_df.groupby(['taxonomic_lineage', 'Functional category description', 'Functional_category'], as_index= False)['number_of_reads'].sum()

func_cat_df.sort_values(['number_of_reads'], ascending=False).head(10)

Now lets use the taxonomic abundance dataframe that we generated previously to extract the 5 most abundant lineages

In [ ]:
# we are extracting the five first five lineages and we already sorted this table so we know that they are the lineages with the most reads
common_lineage=sub_tax_df['taxonomic_lineage'][:5].to_list()
# check if the taxonomic_lineage is one of the five we extracted previously and if it is copy to a new df
common_func_cat_df=func_cat_df[func_cat_df['taxonomic_lineage'].isin(common_lineage)]

common_func_cat_df

And now to plot the abundance of different COG categories by the 5 most abundant genera

In [ ]:
# set plot size
plt.rcParams['figure.figsize'] = [15, 10]

# and now to plot it
ax = sns.barplot(data= common_func_cat_df, x='Functional category description', y='number_of_reads', hue='taxonomic_lineage')

# generate labels
ax.set(xlabel = 'COG functional category',
       ylabel = 'number of reads recruited')


plt.xticks(rotation=90)

# move legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# save plot
plt.savefig(os.path.join(figdir,"COG_functional_categories.pdf"))

Alright this really does not tell us all that much since the Pelagibacteraceae have the most reads being recruited to every functional category, because they are just so abundant in the ocean. Instead of looking at the total number of reads per category lets calculate the percent of reads that are mapping to each category for each taxonomic lineage. This will give us some idea of differences in the genome structure and gene content of each lineage.

In [ ]:

#use our old df to extract the total number of reads that were recruited to each tax lineage
total_reads=sub_tax_df[['taxonomic_lineage', 'number_of_reads']][:5].copy()
total_reads=total_reads.rename(columns={'number_of_reads':'total_number_of_reads'})

#merge the total number of reads with our dataframe and use it to calculate the percent of reads in each functional category
common_func_cat_df=common_func_cat_df.merge(total_reads, on='taxonomic_lineage')
common_func_cat_df['percent_of_tax_reads']=common_func_cat_df['number_of_reads']/common_func_cat_df['total_number_of_reads']*100


# create the barplot
sns.barplot(data= common_func_cat_df, x='Functional category description', y='percent_of_tax_reads', hue='taxonomic_lineage')

# generate labels
plt.xlabel('COG functional category')
plt.xticks(rotation=90)
plt.ylabel('percent of reads recruited per lineage')
plt.title('percent of reads per COG functional category per lineage')

# move legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# make it a bit bigger
plt.rcParams['figure.figsize'] = [15, 10]

# save plot
plt.savefig(os.path.join(figdir,"COG_functional_categories_percent.pdf"))


Now we can start to see some difference between the lienages. The Alphaproteobacteria lineages Pelagibacteraceae and Rhodospirillaceae seem to have a higher percentage of reads (fraction of genome) related to Amino acid transport and metabolism and Energy production and conversion than the other lineages, while the Gammaproteobacteria lineage have the highest fraction of reads recruited to Cell/wall and replication compared to other lineages

The cell below is entirely to demonstrate how to do this for multiple metagenomes, imagine that you have a time series and you want to compare abundance/functions across the time series. You can do this in a notebook and with pandas but it will quickly get out of hand.

In [ ]:

# start timer to check how long this takes 
start = time.time()

# create an empty dataframe that we will populate in this cell
example_master_df=pd.DataFrame(columns=['taxonomic_lineage', 'prokka_product'])

# define the files that I want to summarize I am limiting it to ones where the number ends with a 2 to reduce time
# If I wanted to do it for all files it would be *.txt.gz
infiles=glob.glob('/mnt/storage/lesson_analyses/gorg-classifier/annotations/*2_annotated.txt.gz')
# loop through the files
for file in infiles:
    # extract the base name so that we can use it to rename the column with the metagenome name
    name=os.path.basename(file)
    name,ext=name.split('_')
    # read the file
    data=pd.read_csv(file, compression='gzip', sep='\t')
    # count the number of reads per taxonomic_lineage and product and rename the 
    counts=data[data['status']=='C'].groupby(['taxonomic_lineage', 'prokka_product'], as_index= False)['sequence_id'].count()
    counts=counts.rename(columns={'sequence_id':name})
    
    #merge the new dataframe with the master dataframe
    example_master_df=example_master_df.merge(counts, on=['taxonomic_lineage', 'prokka_product'], how='outer')
    
#timer end    
end = time.time()
print('it took', end - start, 'seconds to load 4 txt files, count the number of reads per function and lineage and \nmerge the counts for each metagenome into a single dataframe')

example_master_df

In [ ]:
example_master_df.to_csv(os.path.join(outdir,'example_master_metagenome_read_recruitment_summary.csv'))

So it took ~21 seconds to generate this df for 4 metagenomes where we recruited 1 million reads per metagenome. While it was doable for this course you can easily see how this would rapidly get out of hand if you tried to increase either 
1. the number of reads (1 billion+ read metagenomes are not uncommon) processing 1 billion reads from each of these four metagenomes would take ~10 hours
1. the number of metagenomes (Imagine you are comparing hundreds of metagenomes from different locations)

or both. 

In order to get around such issues we generally will write a script that will take care of doing the above summation and then once we have that we can just load this final dataframe into our notebook. 


## Further exploration ideas
1. What species has the most reads recruiting to Photosystem I? how about beta-xylanase? or some other gene you are intersted in?
1. What percent of reads recruit to Archaea compared to Bacteria? How about Alphaproteobacteria?
1. As you saw this morning summer metagenomes had higher recruitment to AG-910 than winter metagenomes. We started with a summer sample. Get a winter metagenome and compare  taxonomic and functional differences between the two.
     1. A winter metagenome SRR5720233
         - Hint think about how you are going to ensure that you can differentiate between the two metagenomes
     1. Metadata about other metagenomes can be found in this file (/mnt/storage/lessons/Day3PM_GORG-classifier/bats_metagenomes_of_interest.csv)
         
1. The first number of an EC number indicates the class of an enzyme. Use the EC number to look at the prevelance of different enzyme classes for a single species, between species, or between metagenomes
     - Hint the dictionary below contains the enzyme classes associated with the first digit of the EC number
```
            EC_nums={1: 'Oxidoreductase', 2 : "Transferases", 3 : "Hydrolases", 4 : "Lysases", 5 : "Isomerases", 6 : "Ligases", 7 : "Translocases"}
```

There are also lots of other columns that you can use to investigate these results. Consider using the KO numbers instead.